# Understanding the Python Data Model: Metaclasses and ABCs

## Python Objects
- _"Objects are Python’s abstraction for data. All data in a Python program is represented by objects or by relations between objects"_
- Every `object` has:
    + **Identity**: it never changes once the object has been created (_~= memory address_)
    + **Type**: a reference to another object (its `class` or `type`) which determines the operations that the object supports. In theory, it shouldn't change after creation
    + **Value**: 
        + _usually_ just a mapping from strings to counted object references (`__dict__`). It supports definition of new attributes at any time
        + _but_ some classes are more restricted (e.g. slotted classes, builtins and types defined in CPython extensions)
- The _type_ of an object is another object referenced in the `__class__` attribute
    - _Usually_ `type(foo) == foo.__class__`    

# Python Classes

- A Python class is just another Python object:
    + It stores shared definitions for all instances (dunder and regular methods, class variables)
    + It works as a factory of new instances:
        + Its `__call__()` method creates new objects whose `__class__` attribute points back to the class
        + Default `type.__call__(cls, ...)` implementation calls `cls.__new__(...)` then calls `cls.__init__(...)` (if appropriate)
- Python syntax is translated into _dunder_ function calls looked up in the `instance.__class__` dict (previous level of abstraction of the calling instance)
    + `foo + bar` => `foo.__class__.__add__(foo, bar)`

- The class of a class, a _metaclass_, is also a class (and therefore it essentially follows the same pattern)

 ## Python Data Model

<img src="images/meta-diagram.svg" width="1400">

**Note:** _"The type of all types is `type`"_  is not _strictly_ true. The opposite case is only useful for very narrow use cases


## Metaclasses: Implementing custom class behaviour

- Example: we want to use `A + B` as alternative syntax for `A | B`:
    ```python
    A + B == A | B == Union[A, B]
    ```
- Where should be defined the `__add__()` method?
    ```python
    class A:
        def __add__(self, other):
            ...
    ```
    - **NO!!** This works for `A` instances: `a = A(); b = B(); c = a + b`
    - We need to use a custom metaclass!!
    

In [78]:
from typing import Union

class CustomMeta(type):
    def __add__(cls, other):
        return Union[cls, other]
        
class A(metaclass=CustomMeta):
    pass

class B:
    pass

print(f"{A + B = }")

print("\nCreating A instances...")
a = A()
b = B()
print(a)
# This should fail:
#print(a + b)

A + B = typing.Union[__main__.A, __main__.B]

Creating A instances...


## Implementing custom class behaviour with `type`
- The standard `type` metaclass provides some hooks to customize the class behavior directly in the class, to avoid excesive use of small metaclasses:
    - _(auto-classmethod)_ `object.__init_subclass__(cls, **kwargs)`: Called whenever the containing `class` is subclassed. `cls` is then the new subclass.


In [66]:
class Parent:
    def __init_subclass__(cls, /, **kwargs):
        print(f"__init_subclass__({cls}, {kwargs})")
        super().__init_subclass__(**kwargs)
        if "FOO" not in cls.__dict__:
            cls.FOO = "Default!!"
        
class Child1(Parent):
    FOO = 42

class Child2(Child1):
    pass

print(f"\n{Child1.FOO = }, {Child2.FOO = }")

__init_subclass__(<class '__main__.Child1'>, {})
__init_subclass__(<class '__main__.Child2'>, {})

Child1.FOO = 42, Child2.FOO = 'Default!!'


## Implementing custom class behaviour with `type`
- `type` metaclass customization hooks:
    - _(auto-classmethod)_ `object.__class_getitem__(cls, key)`: Return an object representing the specialization of a generic class by type arguments found in key.
        + It has lower priority than `__getitem__` in the metaclass
        + It is used mostly for run-time implementation of `Generic` typing annotations

In [58]:
class Parent:
    def __class_getitem__(cls, key):
        if isinstance(key, str):
            return [sub for sub in cls.__subclasses__() if sub.__name__ == key]
        else:
            return cls.__subclasses__()[key]
        
class Child0(Parent): ...
class Child1(Parent): ...
class Child2(Parent): ...

print(f"{Parent[0:-1] = }")

print(f"\n{Parent['Child2'] = }")

Parent[0:-1] = [<class '__main__.Child0'>, <class '__main__.Child1'>]

Parent['Child2'] = [<class '__main__.Child2'>]


## Metaclasses: Implementing custom class behaviour
- Using metaclasses is also possible to override class-related operators like `isinstance()` and `issubclass()`
    + `class.__instancecheck__(cls, instance)`: Return `true` if instance should be considered a (direct or indirect) instance of `class`.


In [65]:
import dataclasses, types

class CustomMeta(type):
    def __instancecheck__(cls, instance):
        if hasattr(cls, "DISCRIMINATING_ATTR"):
            assert isinstance(cls.DISCRIMINATING_ATTR, str)
            return getattr(instance, cls.DISCRIMINATING_ATTR, False) is True
        return False

class HappyClass(metaclass=CustomMeta):
    DISCRIMINATING_ATTR = "happy"

a = types.SimpleNamespace(happy=True)
b = types.SimpleNamespace(value=42)

@dataclasses.dataclass
class Foo:
    happy: bool

foo = Foo(True)

print(f"{isinstance(a, HappyClass) = }")
print(f"{isinstance(b, HappyClass) = }")
print(f"{isinstance(foo, HappyClass) = }")


isinstance(a, HappyClass) = True
isinstance(b, HappyClass) = False
isinstance(foo, HappyClass) = True


## Metaclasses: Implementing custom class behaviour
- Using metaclasses is also possible to override class-related operators like `isinstance()` and `issubclass()`
    + `class.__subclasscheck__(cls, subclass)`: Return `true` if `subclass` should be considered a (direct or indirect) `subclass` of `class`.


In [80]:
class CustomMeta(type):
    def __subclasscheck__(cls, subclass):
        name = cls.__name__.split(".")[-1]
        sub_name = subclass.__name__.split(".")[-1]
        return sub_name.startswith(f"{name}") and sub_name[len(name)].isupper()

class Foo(metaclass=CustomMeta):
    pass

class FooSomething:
    pass

class NotFooSomething:
    pass

print(f"{issubclass(FooSomething, Foo) = }")
print(f"{issubclass(NotFooSomething, Foo) = }")


issubclass(FooSomething, Foo) = True
issubclass(NotFooSomething, Foo) = False


## Metaclasses: customizing class creation    
- Customization can happen at multiple points
- For subclasses of `type`, the simplest way is to use the same mechanism available for customization of instances, since classes are just instances of the metaclasss:
    - `__new__()` -> before the _instance_ (a new class in this case) is created 
    - `__init__()` -> after the instance has been created (weird usage in this case)

In [83]:
class MyMeta(type):
    def __new__(*args, **kwargs):
        print(f"MyType.__new__({args}, {kwargs})")
        return type.__new__(*args, **kwargs)
    
    def __init__(cls, *args, **kwargs):
        print(f"\nMyType.__init__({cls}, {args}, {kwargs})")
        cls.VALUE = 0
        return None
    

class A(metaclass=MyMeta):
    VALUE = 42

class ABC(metaclass=abc.ABCMeta):
    pass
    
class B(abc.ABC): pass
    
B.__class__

#rint(f"\n{A.VALUE = }")

MyType.__new__((<class '__main__.MyMeta'>, 'A', (), {'__module__': '__main__', '__qualname__': 'A', 'VALUE': 42}), {})

MyType.__init__(<class '__main__.A'>, ('A', (), {'__module__': '__main__', '__qualname__': 'A', 'VALUE': 42}), {})
MyType.__new__((<class '__main__.MyMeta'>, 'B', (<class '__main__.A'>,), {'__module__': '__main__', '__qualname__': 'B'}), {})

MyType.__init__(<class '__main__.B'>, ('B', (<class '__main__.A'>,), {'__module__': '__main__', '__qualname__': 'B'}), {})


__main__.MyMeta

## Metaclasses: customizing class creation
- `class` keyword is just syntatic sugar to call the metaclass
    ```python
    class MyType(A, B, C, metaclass=MyMeta, kwarg2=33):
        VALUE = 42        
        def x(self, b): return ...
    ```
    is roughly equivalent to ([3.3.3.1. Metaclasses](https://docs.python.org/3/reference/datamodel.html#metaclasses)):
    ```python
    # MRO entries are resolved
    bases = solve_mro_entries(A, B, C)  # For regular bases == (A, B, C)
    # the appropriate metaclass is determined
    kwargs = dict(metaclass=MyMeta, kwarg2=33)
    metaclass = kwargs.pop("metaclass", type)
    # the class namespace is prepared
    body_ns = metaclass.__prepare__("MyType", bases, **kwargs) # == dict()
    # the class body is executed
    exec(SOURCE_LINES[1:4], globals(), body_ns)
    # the class object is created
    new_class = metaclass("MyType", bases, body_ns, **kwargs)
    ```
    

In [75]:
class FooMeta(type):
    def __new__(mcls, name, bases, ns, **kwargs):
        filtered_ns = {key: value for key, value in ns.items() if key not in ["foo", "bar"]}
        #print(ns, filtered_ns)
        return type(name, bases, filtered_ns, **kwargs)

class Foo(metaclass=FooMeta):
    def foo(self):
        return 42
    
    def bar(self):
        return 42
    
    def something(self):
        return 42
    
print(f"{vars(Foo).keys() = }")

vars(Foo).keys() = dict_keys(['__module__', 'something', '__dict__', '__weakref__', '__doc__'])
